In [1]:
from pymongo import MongoClient
from pprint import pprint

# Conectando ao MongoDB local
client = MongoClient("mongodb://localhost:27017/")
db = client["vendas"]
colecao = db["clientes"]

In [6]:
import pandas as pd
from pymongo import MongoClient

# Conexão com MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["vendas"]
colecao = db["clientes"]

# Buscar documentos e transformar em DataFrame
dados = list(colecao.find({}, { "_id": 0 }))  # Remove o campo _id
df = pd.DataFrame(dados)

df

,nome,cpf,vip
0,João Lima,123.456.789-00,True
1,João Lima,123.456.789-00,True


In [7]:
from pymongo import MongoClient
from datetime import datetime

client = MongoClient("mongodb://localhost:27017/")
db = client["vendas"]
notas = db["notafiscal"]

nota = {
    "cliente": {
        "cpf": "123.456.789-00",
        "nome": "João Lima"
    },
    "data": datetime.now(),
    "itens": [
        { "produto": "Notebook Gamer", "quantidade": 1, "preco_unitario": 4899.90 },
        { "produto": "Mouse", "quantidade": 2, "preco_unitario": 89.90 }
    ],
    "total": 4899.90 + 2 * 89.90,
    "pagamento": {
        "forma": "cartao",
        "parcelas": 3
    }
}

resultado = notas.insert_one(nota)
print("🧾 Nota fiscal inserida com ID:", resultado.inserted_id)

🧾 Nota fiscal inserida com ID: 686ae8f2cb4a8377f4aea50e


In [9]:
from pprint import pprint

for nf in notas.find({}, { "_id": 0 }):
    pprint(nf)

dados = list(colecao.find({}, { "_id": 0 }))  # Remove o campo _id
df = pd.DataFrame(dados)



{'cliente': {'cpf': '123.456.789-00', 'nome': 'João Lima'},
 'data': datetime.datetime(2025, 7, 6, 18, 21, 54, 290000),
 'itens': [{'preco_unitario': 4899.9,
            'produto': 'Notebook Gamer',
            'quantidade': 1},
           {'preco_unitario': 89.9, 'produto': 'Mouse', 'quantidade': 2}],
 'pagamento': {'forma': 'cartao', 'parcelas': 3},
 'total': 5079.7}


In [10]:
import pandas as pd
from pymongo import MongoClient

client = MongoClient("mongodb://localhost:27017/")
db = client["vendas"]
notas = db["notafiscal"]

# Buscar documentos e transformar em DataFrame
dados = list(notas.find({}, { "_id": 0 }))
df = pd.DataFrame(dados)

# Mostrar resultado
df

,cliente,data,itens,total,pagamento
0,"{'cpf': '123.456.789-00', 'nome': 'João Lima'}",2025-07-06 18:21:54.290,"[{'produto': 'Notebook Gamer', 'quantidade': 1...",5079.7,"{'forma': 'cartao', 'parcelas': 3}"


In [2]:
print("👥 Clientes cadastrados:")
for cliente in colecao.find({}, { "_id": 0 }):
    pprint(cliente)

👥 Clientes cadastrados:
{'cpf': '123.456.789-00', 'nome': 'João Lima', 'vip': True}
{'cpf': '123.456.789-00', 'nome': 'João Lima', 'vip': True}


In [3]:
# Agrupa por CPF e identifica documentos duplicados
duplicados = colecao.aggregate([
    {
        "$group": {
            "_id": "$cpf",
            "ids": { "$addToSet": "_id" },
            "count": { "$sum": 1 }
        }
    },
    {
        "$match": { "count": { "$gt": 1 } }
    }
])

# Apaga todos os documentos duplicados, mantendo um
for doc in duplicados:
    ids = doc["ids"]
    ids.pop(0)  # mantém o primeiro
    colecao.delete_many({ "_id": { "$in": ids } })

print("✅ Duplicados removidos com sucesso!")

✅ Duplicados removidos com sucesso!
